In [ ]:
import os
import glob
import argparse
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [ ]:
def test_total(dataset_dir, input_width, input_height, model):
    test_datagen = ImageDataGenerator(rescale=1./255)
    batch_size = 10
    test_generator = test_datagen.flow_from_directory(
        dataset_dir,
        target_size=(input_width,input_height),
        batch_size=batch_size,
        class_mode='binary')
    test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
    return test_loss, test_acc

def test_each(dataset_dir, input_width, input_height, model):
    result = []
    for path in glob.glob('{}/**/*.jpeg'.format(dataset_dir)):
        x = img_to_array(load_img(path, target_size=(input_width, input_height)))
        score = model.predict(np.expand_dims(x[:,:,0:3]/255, axis=0), verbose=1)
        result.append([path, score[0][0]])

    return np.array(result)


In [ ]:
model_dir = 'models/hoo' 
dataset_dir = 'dataset/test'
output_file = 'out/hoo/result.csv'
img_size = 480
model = models.load_model(model_dir)
#test_loss, test_acc = test_total(dataset_dir, img_size, img_size, model)
#print('test loss: {}, accuracy: {}'.format(test_loss, test_acc))
result = test_each(dataset_dir, img_size, img_size, model)
np.savetxt(output_file, result, delimiter=',', fmt='%s')
